# Gait generation

The state and control vectors $\textbf{x}$ and $\textbf{u}$ are defined as follows:

$$
\begin{equation*}
\textbf{X} = \begin{bmatrix}
    x & y & z & \dot{x} &\dot{y} &\dot{z} 
    \end{bmatrix}^T
\end{equation*}
$$

$$
\begin{equation*}
    \textbf{u} = \begin{bmatrix}
    x_z & y_z & F_z
    \end{bmatrix}^T
\end{equation*}
$$


The dynamic equation for the system will be 

$$
\begin{equation*}
    \begin{bmatrix}
    \dot{X}  
    \end{bmatrix} 
    =
    \begin{bmatrix}
    \dot{x} \\ 
    \dot{y} \\ 
    \dot{z} \\ 
    \frac{(x-x_z)F_z}{mz} \\ 
    \frac{(y-y_z)F_z}{mz} \\ 
    \frac{F_z}{m}-g
    \end{bmatrix}
\end{equation*}
$$


where $\textbf{m}$ and $\textbf{g}$ are mass and the acceleration of gravity. In the following case, I will set $\textbf{m}$ to be 1.






In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function

In [4]:
import numpy as np
import theano.tensor as T
import matplotlib.pyplot as plt

In [5]:
from ilqr import iLQR
from ilqr.cost import PathQRCost
from ilqr.dynamics import tensor_constrain
from ilqr.dynamics import AutoDiffDynamics

In [6]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    J_hist.append(J_opt)
    info = "converged" if converged else ("accepted" if accepted else "failed")
    print("iteration", iteration_count, info, J_opt)

In [7]:
x_inputs = [
    T.dscalar("x"),
    T.dscalar("y"),
    T.dscalar("z"),
    T.dscalar("x_dot"),
    T.dscalar("y_dot"),
    T.dscalar("z_dot"),
]

u_inputs = [
    T.dscalar("x_zmp"),
    T.dscalar("y_zmp"),
    T.dscalar("F_z"),
]

t = T.dscalar("t")


dt = 0.01  # Discrete time step.
g = 9.81  # g.

'''
# nonlinear dynamics.
def nonldyn(x_inputs, u_inputs):
    x_inputs_dot[0]=x_inputs[3],
    x_inputs_dot[1]=x_inputs[4],
    x_inputs_dot[2]=x_inputs[5],
    x_inputs_dot[3]=(x_inputs[0]-u_inputs[0])*u_inputs[2]/x_inputs[2],
    x_inputs_dot[4]=(x_inputs[1]-u_inputs[1])*u_inputs[2]/x_inputs[2],
    x_inputs_dot[5]=u_inputs[2]-g,
    return x_inputs_dot

# discritize with RK4
x_inputs_dot1=nonldyn(x_inputs,u_inputs)
x_inputs_dot2=nonldyn(x_inputs+.5*dt*x_inputs_dot1,u_inputs)
x_inputs_dot3=nonldyn(x_inputs+.5*dt*x_inputs_dot2,u_inputs)
x_inputs_dot4=nonldyn(x_inputs+dt*x_inputs_dot3,u_inputs)

f = x_inputs+(dt/6)*(x_input_dot1+2*x_input_dot2+2*x_input_dot3+x_input_dot4)
'''


# Discrete dynamics model definition.
f = T.stack([
    x_inputs[0] + x_inputs[3] * dt,
    x_inputs[1] + x_inputs[4] * dt,
    x_inputs[2] + x_inputs[5] * dt,
    x_inputs[3] + (x_inputs[0]-u_inputs[0])*u_inputs[2]/x_inputs[2] * dt,
    x_inputs[4] + (x_inputs[1]-u_inputs[1])*u_inputs[2]/x_inputs[2] * dt,
    x_inputs[5] + (u_inputs[2]-g) * dt,
])


In [20]:
T = 100
state_size = 6
action_size = 3

Q = np.eye(state_size)
R = np.eye(action_size)
a = np.array([10,13.626,16.897,20.551,25.318,31.351,37.813,43.768,48.33,51.378,53.388,54.85,56.258,58.103,60.877,65.072,70.774,77.131,83.161,87.906,91.1,93.2,94.699,96.089,97.861,100.51,104.52,110.09,116.42,122.54,127.46,130.81,133.01,134.55,135.92,137.63,140.16,143.99,149.42,155.71,161.9,167,170.5,172.8,174.39,175.76,177.41,179.82,183.49,188.75,195,201.25,206.51,210.18,212.59,214.24,215.61,217.2,219.5,223,228.1,234.29,240.58,246.01,249.84,252.37,254.08,255.45,256.99,259.19,262.54,267.46,273.58,279.91,285.48,289.49,292.14,293.91,295.3,296.8,298.9,302.09,306.84,312.87,319.23,324.93,329.12,331.9,333.74,335.15,336.61,338.62,341.67,346.23,352.19,358.65,364.68,369.45,373.1,376.37,5,5.1312,6.0498,8.5429,13.388,20.545,28.577,35.909,41.049,43.765,44.82,45,44.908,44.116,41.836,37.285,30.322,22.31,14.822,9.39,6.44,5.2396,5.0014,5.0617,5.7373,7.8125,12.075,18.823,26.797,34.421,40.14,43.333,44.689,44.995,44.961,44.392,42.512,38.531,32.016,24.1,16.36,10.362,6.9166,5.3955,5.0115,5.0225,5.4939,7.1901,10.898,17.162,25,32.838,39.102,42.81,44.506,44.977,44.988,44.605,43.083,39.638,33.64,25.9,17.984,11.469,7.4883,5.6075,5.0389,5.0049,5.311,6.667,9.86,15.579,23.203,31.177,37.925,42.187,44.263,44.938,44.999,44.76,43.56,40.61,35.178,27.69,19.678,12.715,8.1637,5.8843,5.0922,5.0002,5.18,6.2346,8.9514,14.091,21.423,29.455,36.612,41.457,43.95,44.869,80.29,80.01,79.805,79.697,79.795,80.206,80.627,80.657,80.413,80.178,80.022,79.918,79.832,79.747,79.69,79.78,80.159,80.598,80.676,80.447,80.206,80.042,79.934,79.848,79.762,79.697,79.76,80.107,80.561,80.686,80.472,80.223,80.051,79.939,79.851,79.765,79.696,79.737,80.053,80.521,80.696,80.507,80.253,80.072,79.955,79.867,79.781,79.706,79.726,80.006,80.476,80.697,80.532,80.273,80.083,79.96,79.87,79.784,79.707,79.709,79.957,80.428,80.696,80.564,80.304,80.106,79.978,79.886,79.8,79.719,79.705,79.917,80.377,80.686,80.588,80.326,80.118,79.984,79.889,79.804,79.721,79.694,79.874,80.323,80.674,80.617,80.359,80.143,80.002,79.905,79.819,79.735,79.695,79.842,80.272,80.657,80.624,80.33,80.044,79.826
])
comref=np.reshape(a,(100,3),order='F')
x1t3=comref.T
x4t6=np.zeros((3,100))
xinitial_trans=np.array([10,5,80,0,0,0])
xinitial=xinitial_trans.reshape(-1, 1)
x_T=np.r_[x1t3,x4t6]
x_path=np.c_[xinitial,x_T].T
upath3=np.full((100,1),10)
upath1t2=np.c_[x1t3[0,:],x1t3[1,:]]
u_path= np.c_[upath1t2,upath3]


cost = PathQRCost(Q, R, x_path, u_path=u_path)


In [21]:
N = 100  # Number of time steps in trajectory.
x0 = np.array([10, 5, 8, 0, 0, 0])  # Initial state.

# Random initial action path.
us_init = np.random.uniform(-100, 100, (N, dynamics.action_size))

In [22]:
J_hist = []
ilqr = iLQR(dynamics, cost, N)
xs, us = ilqr.fit(x0, us_init, on_iteration=on_iteration)

iteration 0 accepted 6086095.08585
iteration 1 accepted 4729373.32425
iteration 2 accepted 4539263.06136
iteration 3 accepted 4471525.12823
iteration 4 accepted 4336416.589
iteration 5 accepted 4256460.68591
iteration 6 accepted 4170427.93842
iteration 7 accepted 4111728.91303
iteration 8 accepted 4059545.00359
iteration 9 accepted 4023699.50625
iteration 10 accepted 3994146.68361
iteration 11 accepted 3974648.06278
iteration 12 accepted 3959451.38431
iteration 13 accepted 3950242.96057
iteration 14 accepted 3943414.17226
iteration 15 accepted 3939602.21337
iteration 16 accepted 3936619.85934
iteration 17 accepted 3934884.97578
iteration 18 accepted 3933345.33354
iteration 19 accepted 3932373.32523
iteration 20 accepted 3931450.48724
iteration 21 accepted 3930836.47366
iteration 22 accepted 3930243.41313
iteration 23 accepted 3929835.37554
iteration 24 accepted 3929442.24114
iteration 25 accepted 3929164.87716
iteration 26 accepted 3928900.08763
iteration 27 accepted 3928709.35557
iter

In [16]:
xs

array([[  1.00000000e+01,   5.00000000e+00,   8.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.00000000e+01,   5.00000000e+00,   8.00000000e+00,
          6.38210488e+01,   1.21447053e+01,   2.54801761e+00],
       [  1.06382105e+01,   5.12144705e+00,   8.02548018e+00,
          9.16657831e+01,   1.80564489e+01,   4.36361780e+00],
       [  1.15548683e+01,   5.30201154e+00,   8.06911635e+00,
          1.09009010e+02,   2.21396201e+01,   5.90965837e+00],
       [  1.26449584e+01,   5.52340774e+00,   8.12821294e+00,
          1.21171620e+02,   2.51008204e+01,   7.30282433e+00],
       [  1.38566746e+01,   5.77441595e+00,   8.20124118e+00,
          1.30046147e+02,   2.69598262e+01,   8.59216049e+00],
       [  1.51571361e+01,   6.04401421e+00,   8.28716279e+00,
          1.36413024e+02,   2.75816614e+01,   9.79745162e+00],
       [  1.65212663e+01,   6.31983082e+00,   8.38513730e+00,
          1.40842195e+02,   2.70225690e+01,   1.09314790e+01],


In [18]:
us

array([[-182.94999863,  -31.71705361,  264.61176142],
       [-106.4016581 ,  -19.71335355,  191.3700184 ],
       [ -74.01861599,  -14.80957939,  164.41405761],
       [ -54.25600416,  -10.72079752,  149.12659523],
       [ -39.34593535,   -5.36746803,  138.74361676],
       [ -26.20520519,    1.86168353,  130.33911232],
       [ -14.63301517,    9.80441268,  123.21273611],
       [  -4.98490488,   17.04642568,  117.36594654],
       [   2.02143085,   22.09980289,  113.12159424],
       [   6.53111699,   24.76512896,  110.40540172],
       [   9.29464994,   25.85631107,  108.75135458],
       [  11.03673812,   26.21316874,  107.72617853],
       [  12.43324851,   26.48298129,  106.93936985],
       [  14.05370251,   26.2438871 ,  106.22217144],
       [  16.48390458,   24.71256754,  105.42603771],
       [  20.31055325,   21.08884061,  104.38008723],
       [  25.7256108 ,   15.18797167,  102.91775388],
       [  31.95048354,    8.29397526,  101.0812573 ],
       [  37.90945468,    1.